# Webscraping with Selenium
*****
When the data that you want exists on a website with heavy JavaScript and requires interaction from the user, `BeautifulSoup` will not be enough. This is when you need a webdriver. One of the most popular webdrivers is [Selenium](http://www.seleniumhq.org/projects/webdriver/). Selenium is commonly used in industry to automate testing of the user experience, but it can also interact with content to collect data that are difficult to get otherwise.

This lesson is a short introduction to the Selenium webdriver. It includes:

1. Launching the webdriver
2. Navigating the browser
3. Collecting generated data
4. Exporting data to CSV

Let's first import the necessary Python libraries:

In [1]:
from selenium import webdriver  # powers the browser interaction
from selenium.webdriver.support.ui import Select  # selects menu options
from pyvirtualdisplay import Display
from selenium import webdriver
from bs4 import BeautifulSoup  # to parse HTML
import csv  # to write CSV
import pandas  # to see CSV

Selenium actually uses our web browser, and since the JupyterHub doesn't come with Firefox, we'll download the binaries:

In [ ]:
# download firefox binaries
!wget http://ftp.mozilla.org/pub/firefox/releases/54.0/linux-x86_64/en-US/firefox-54.0.tar.bz2
    
# untar binaries
!tar xvjf firefox-54.0.tar.bz2

We also need the webdriver for Firefox that allows Selenium to interact directly with the browser through the code we write. We can download the `geckodriver` for Firefox from the github page:

In [ ]:
# download geckodriver
!wget https://github.com/mozilla/geckodriver/releases/download/v0.17.0/geckodriver-v0.17.0-linux64.tar.gz
    
# untar geckdriver
!tar xzvf geckodriver-v0.17.0-linux64.tar.gz

# 1. Launching the webdriver

Since we are in different environment and we can't use our regular graphical desktop, we need to tell Python to start a virutal display, onto which `Selenium` can project the Firefox web browser (though we won't actually see it).

In [3]:
display = Display(visible=0, size=(1024, 768))
display.start()

<Display cmd_param=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1024x768x24', ':1001'] cmd=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1024x768x24', ':1001'] oserror=None return_code=None stdout="None" stderr="None" timeout_happened=False>

Now we can initialize the `Selenium` web driver, giving it the path to the Firefox binary code and the driver:

In [4]:
# setup driver
!chmod -R +x firefox
!chmod a+x geckodriver
driver = webdriver.Firefox(firefox_binary='./firefox/firefox', executable_path="./geckodriver")  # relative path where we just downloaded it

You can navigate `Selenium` to a URL by using the `get` method, exactly the same way we used the `requests.get` before:

In [17]:
driver.get("http://www.google.com")
print(driver.page_source)

Cool, right? You can see Google in your browser now. Let's go look at some [West Bengal State](http://wbsec.gov.in/) election results:

# 2. Navigating the browser

In [18]:
# go results page
driver.get("http://wbsec.gov.in/(S(eoxjutirydhdvx550untivvu))/DetailedResult/Detailed_gp_2013.aspx")

### Zilla Parishad

Similar to `BeautifulSoup`, `Selenium` has methods to find elements on a webpage. We can use the method `find_element_by_name` to find an element on the page by its name.

In [19]:
# find "district" drop down menu
district = driver.find_element_by_name("ddldistrict")

In [20]:
district

<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="30c9b0c4-7418-4b87-9ef3-8b68c5c9407a", element="4a2cfd07-2448-4d57-8a34-8bca9d03a66e")>

Now if we want to get the different options in this drop down, we can do the same. You'll notice that each name is associated with a unique value. Since we're getting multiple elements here, we'll use `find_elements_by_tag_name`

In [21]:
# find options in "disrict" drop down
district_options = district.find_elements_by_tag_name("option")

print(district_options[1].get_attribute("value"))
print(district_options[1].text)

1
Bankura


Now we'll make a dictionary associating each name with its value.

In [22]:
d_options = {option.text.strip(): option.get_attribute("value") for option in district_options if option.get_attribute("value").isdigit()}
print(d_options)

{'Kolkata-North': '20', 'Nadia': '13', 'Kolkata-South': '22', 'Jalpaiguri': '7', 'Purba Medinipur': '11', 'Alipurduar': '21', 'Bankura': '1', 'Birbhum': '3', 'Uttar Dinajpur': '18', 'Paschim Medinipur': '10', 'Purulia': '14', 'Cooch Behar': '8', 'Malda': '9', 'Hooghly': '6', 'Murshidabad': '12', 'Howrah': '5', 'Dakshin Dinajpur': '17', 'Darjeeling': '4', 'Burdwan': '2', 'Siliguri MP': '19', 'South 24-Parganas': '16', 'North 24-Parganas': '15'}


We can then select a district by using its name and our dictionary. First we'll make our own function using `Selenium`'s `Select`, and then we'll call it on "Bankura".

In [23]:
district_select = Select(district)
district_select.select_by_value(d_options["Bankura"])

You should have seen the dropdown menu select 'Bankura' by running the previous cell.

### Panchayat Samity

We can do the same as we did above to find the different blocks.

In [24]:
# find the "block" drop down
block = driver.find_element_by_name("ddlblock")

In [25]:
# get options
block_options = block.find_elements_by_tag_name("option")

print(block_options[1].get_attribute("value"))
print(block_options[1].text)

1
BANKURA-I


In [26]:
b_options = {option.text.strip(): option.get_attribute("value") for option in block_options if option.get_attribute("value").isdigit()}
print(b_options)

{'TALDANGRA': '9', 'SIMLAPAL': '10', 'BARJORA': '7', 'BANKURA-II': '2', 'CHHATNA': '3', 'PATRASAYAR': '21', 'ONDA': '8', 'MEJHIA': '5', 'JOYPUR': '18', 'RANIBUNDH': '16', 'RAIPUR': '14', 'SALTORA': '4', 'SARENGA': '15', 'INDUS': '22', 'KOTULPUR': '19', 'GANGAJALGHATI': '6', 'KHATRA': '11', 'SONAMUKHI': '20', 'BANKURA-I': '1', 'HIRBUNDH': '12', 'BISHNUPUR': '17', 'INDPUR': '13'}


In [27]:
block_select = Select(block)
block_select.select_by_value(b_options["BANKURA-I"])

Great! One dropdown menu to go.

### Gram Panchayat

In [28]:
# get options
gp = driver.find_element_by_name("ddlgp")
gp_options = gp.find_elements_by_tag_name("option")

print(gp_options[1].get_attribute("value"))
print(gp_options[1].text)

1
ANCHURI


In [29]:
gp_options = {option.text.strip(): option.get_attribute("value") for option in gp_options if option.get_attribute("value").isdigit()}
print(gp_options)

{'KALAPATHAR': '5', 'JAGADALLA-I': '3', 'ANDHARTHOLE': '2', 'JAGADALLA-II': '4', 'KENJAKURA': '6', 'ANCHURI': '1'}


In [30]:
gp_select = Select(gp)
gp_select.select_by_value(gp_options["ANCHURI"])

Once we selected the last dropdown menu parameter, the website automatically generate a table below. This table could not have been called up by a URL, as you can see that the URL in the browser did not change. This is why `Selenium` is so helpful.

# 3. Collecting generated data

Now that the table has been rendered, it exists as HTML in our page source. If we wanted to, we could send this to `BeautifulSoup` using the `driver.page_source` method to get the text. But we can also use `Selenium`'s parsing methods.

First we'll identify it by its CSS selector, and then use the `get_attribute` method.

In [31]:
soup = BeautifulSoup(driver.page_source, 'html5lib')

In [32]:
# get the html for the table
table = soup.select('#DataGrid1')[0]

First we'll get all the rows of the table using the `tr` selector.

In [33]:
# get list of rows
rows = [row for row in table.select("tr")]

But the first row is the header so we don't want that.

In [34]:
rows = rows[1:]

Each cell in the row corresponds to the data we want.

In [35]:
rows[0].select('td')

[<td style="font-weight:bold;">
                         <span id="DataGrid1_ctl02_txtseat">I/1</span>
                         <br/>
                         
                         
                         <span id="DataGrid1_ctl02_txtReserve" style="color:#993366;">WOMAN</span>
                         </td>, <td>
                         <span id="DataGrid1_ctl02_lblelectors">830</span>
                         </td>, <td>
                         <span id="DataGrid1_ctl02_lblVPolled">646</span>
                         </td>, <td>
                         <span id="DataGrid1_ctl02_lblVRejected">19</span>
                         </td>, <td>1</td>, <td>RITA CHATTOPADHYAY</td>, <td>IND</td>, <td>301</td>]

Now it's just a matter of looping through the rows and getting the information we want from each one.

In [36]:
data = []
for row in rows:
    d = {}
    seat_names = row.select('td')[0].find_all("span")
    d['seat'] = ' '.join([x.text for x in seat_names])
    d['electors'] = row.select('td')[1].text.strip()
    d['polled'] = row.select('td')[2].text.strip()
    d['rejected'] = row.select('td')[3].text.strip()
    d['osn'] = row.select('td')[4].text.strip()
    d['candidate'] = row.select('td')[5].text.strip()
    d['party'] = row.select('td')[6].text.strip()
    d['secured'] = row.select('td')[7].text.strip()
    data.append(d)

In [37]:
print(data[1])

{'osn': '2', 'party': 'BJP', 'candidate': 'RINA GHOSH', 'electors': '', 'rejected': '', 'secured': '57', 'seat': '', 'polled': ''}


You'll notice that some of the information, such as total electors, is not supplied for each canddiate. This code will add that information for the candidates who don't have it.

In [38]:
i = 0
while i < len(data):
    if data[i]['seat']:
        seat = data[i]['seat']
        electors = data[i]['electors']
        polled = data[i]['polled']
        rejected = data[i]['rejected']
        i = i+1
    else:
        data[i]['seat'] = seat
        data[i]['electors'] = electors
        data[i]['polled'] = polled
        data[i]['rejected'] = rejected
        i = i+1

In [39]:
data

[{'candidate': 'RITA CHATTOPADHYAY',
  'electors': '830',
  'osn': '1',
  'party': 'IND',
  'polled': '646',
  'rejected': '19',
  'seat': 'I/1 WOMAN',
  'secured': '301'},
 {'candidate': 'RINA GHOSH',
  'electors': '830',
  'osn': '2',
  'party': 'BJP',
  'polled': '646',
  'rejected': '19',
  'seat': 'I/1 WOMAN',
  'secured': '57'},
 {'candidate': 'RINA PATRA',
  'electors': '830',
  'osn': '3',
  'party': 'AIFB',
  'polled': '646',
  'rejected': '19',
  'seat': 'I/1 WOMAN',
  'secured': '269'},
 {'candidate': 'ANIMA BAURI',
  'electors': '659',
  'osn': '1',
  'party': 'AITC',
  'polled': '528',
  'rejected': '39',
  'seat': 'II/2 SC WOMAN',
  'secured': '210'},
 {'candidate': 'APARNA MANDAL',
  'electors': '659',
  'osn': '2',
  'party': 'CPIM',
  'polled': '528',
  'rejected': '39',
  'seat': 'II/2 SC WOMAN',
  'secured': '279'},
 {'candidate': 'ABDUL MOTALEB MOLLA',
  'electors': '1066',
  'osn': '1',
  'party': 'AITC',
  'polled': '747',
  'rejected': '17',
  'seat': 'III/3 BC',

# 4. Exporting data to CSV

We can then loop through all the combinations of the dropdown menu we want, collect the information from the generated table, and append it to the `data` list. Once we're done, we can write it to a CSV.

In [ ]:
header = data[0].keys()

with open('WBS-table.csv', 'w') as output_file:
    dict_writer = csv.DictWriter(output_file, header)
    dict_writer.writeheader()
    dict_writer.writerows(data)
    
pandas.read_csv('WBS-table.csv')